[1] Setting up Environment and Install Requirements

In [ ]:
import os
import sys
# Set Path
from google.colab import drive
drive.mount('/content/drive')
path_wd = '/content/drive/MyDrive/Github/Content'
os.chdir(path_wd)

!pip install -r sources/ChinaFilm/requirements.txt
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')